In [1]:
from typing import List, Dict, Set, Tuple
def minmax_scale(scores: Dict[str, float], scale_min=0.0, scale_max=10.0) -> Dict[str, float]:
    values = list(scores.values())
    min_val, max_val = min(values), max(values)
    if min_val == max_val:
        return {k: scale_min for k in scores}  # 모든 점수가 같으면 최소값으로 고정

    return {
        k: scale_min + (v - min_val) / (max_val - min_val) * (scale_max - scale_min)
        for k, v in scores.items()
    }

In [2]:
import re
import os
import hashlib
import pickle
from typing import List, Dict

import torch
from transformers import AutoModel, AutoTokenizer


class RerankModel:
    def __init__(self, model_name: str, cache_dir: str, device: str = "cuda"):
        """모델과 토크나이저를 로드하여 초기화"""
        self.model = AutoModel.from_pretrained(model_name).to(device)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.device = device
        self.cache_dir = cache_dir
        os.makedirs(self.cache_dir, exist_ok=True)

    def _get_cache_path(self, text: str) -> str:
        """텍스트를 기반으로 캐시 파일 경로 생성"""
        key = hashlib.md5(text.encode()).hexdigest()
        return os.path.join(self.cache_dir, f"{key}.pkl")

    def _load_embedding_from_cache(self, text: str) -> torch.Tensor:
        """캐시에서 임베딩 로드"""
        path = self._get_cache_path(text)
        if os.path.exists(path):
            with open(path, "rb") as f:
                return pickle.load(f)
        return None

    def _save_embedding_to_cache(self, text: str, embedding: torch.Tensor):
        """임베딩을 캐시에 저장"""
        path = self._get_cache_path(text)
        with open(path, "wb") as f:
            pickle.dump(embedding.cpu(), f)

    def _embed(self, text: str) -> torch.Tensor:
        """텍스트를 의미 벡터로 변환"""
        inputs = self.tokenizer(text, padding=True, truncation=True, return_tensors="pt").to(self.device)
        with torch.no_grad():
            hidden = self.model(**inputs).last_hidden_state
            mask = inputs["attention_mask"].unsqueeze(-1)
            pooled = (hidden * mask).sum(dim=1) / mask.sum(dim=1)
        return pooled[0]

    def _get_embedding(self, text: str, is_query: bool = False) -> torch.Tensor:
        """임베딩 반환 (문서는 캐싱, 쿼리는 계산)"""
        if is_query:
            return self._embed(text)

        cached = self._load_embedding_from_cache(text)
        if cached is not None:
            return cached.to(self.device)

        emb = self._embed(text)
        self._save_embedding_to_cache(text, emb)
        return emb

    def rerank(self, query: str, documents: List[str]) -> Dict[str, float]:
        """쿼리와 문서들의 유사도를 계산해 점수 반환"""
        query_embedding = self._get_embedding(query, is_query=True).unsqueeze(0)
        doc_embeddings = [self._get_embedding(doc).unsqueeze(0) for doc in documents]
        doc_tensor = torch.cat(doc_embeddings, dim=0)

        cos_scores = torch.nn.functional.cosine_similarity(query_embedding, doc_tensor)
        return {doc: score.item() for doc, score in zip(documents, cos_scores)}

    def cache_embeddings(self, texts: List[str], max_length: int = 512):
        """전체 문서에 대해 의미 임베딩 캐싱"""
        skipped, cached = 0, 0
        for text in texts:
            text = text[:max_length]
            path = self._get_cache_path(text)
            if os.path.exists(path):
                skipped += 1
                continue
            emb = self._embed(text)
            self._save_embedding_to_cache(text, emb)
            cached += 1
        print(f"✅ 의미 임베딩 캐싱 완료 | 새로 캐싱: {cached}개 | 스킵: {skipped}개")




In [3]:

class TokenizerWrapper:
    def __init__(self, engine="kiwi"):
        if engine == "kiwi":
            from kiwipiepy import Kiwi
            self.tokenizer = Kiwi()
            self.mode = "kiwi"
        elif engine == "okt":
            from konlpy.tag import Okt
            self.tokenizer = Okt()
            self.mode = "okt"
        else:
            raise ValueError(f"지원되지 않는 분석기: {engine}")

    def tokenize_korean(self, text: str, use_bigrams: bool = True) -> List[str]:
        # ✅ 명사만 추출
        if self.mode == "kiwi":
            tokens = [token.form for token in self.tokenizer.tokenize(text) if token.tag.startswith("NN")]
        elif self.mode == "okt":
            tokens = self.tokenizer.nouns(text)

        # ✅ 불용어 제거
        stopwords = {"에서", "는", "은", "이", "가", "하", "어야", "에", "을", "를", "도", "로", "과", "와", "의", "?", "다"}
        tokens = [t for t in tokens if t not in stopwords]

        # ✅ 바이그램 추가
        if use_bigrams:
            bigrams = [tokens[i] + tokens[i+1] for i in range(len(tokens) - 1)]
            tokens += bigrams

        return tokens



In [37]:
import re
import os
import json
import pickle
import logging
from typing import List, Dict, Set, Tuple

from tqdm import tqdm
from tqdm.asyncio import tqdm_asyncio
from langchain_chroma import Chroma
from langchain.schema import Document
from rank_bm25 import BM25Okapi 
from more_itertools import chunked

import aiofiles

# 로컬 임포트
# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


class RetrieverError(Exception):
    """Retriever에서 발생하는 예외를 위한 기본 클래스"""
    pass

class ChunkLoadingError(RetrieverError):
    """JSON 청크 로딩 실패 시 발생하는 예외"""
    pass


class Retriever:
    def __init__(self,
                 meta_df=None,
                 embedder=None,
                 reranker: RerankModel =None,
                 tokenizer=None,
                 persist_directory=None,
                 rerank_max_length=512,
                 bm25_weight=0.5,
                 rerank_weight=0.5,
                 bm25_path="bm25_index.pkl",
                 debug_mode=False
                 ):
        self.meta_df = meta_df
        self.embedder = embedder
        self.reranker = reranker
        self.tokenizer = tokenizer or TokenizerWrapper("kiwi")
        self.persist_directory = persist_directory
        self.rerank_max_length = rerank_max_length

        self.bm25_weight = bm25_weight
        self.rerank_weight = rerank_weight

        self.db = None
        self.bm25 = None
        self.bm25_ready = False
        self.bm25_path = bm25_path
        self.documents = []
        self.debug_mode = debug_mode

        self.last_scores = {}

    def set_weights(self, bm25_weight: float, rerank_weight: float):
        self.bm25_weight = bm25_weight
        self.rerank_weight = rerank_weight
        logging.info(f"🔧 가중치 설정됨 | BM25: {bm25_weight} | Rerank: {rerank_weight}")

    def get_doc_key(self, doc: Document) -> str:
        chunk_id = doc.metadata.get("chunk_id")
        if chunk_id:
            return chunk_id
        return str(hash(doc.page_content.strip()))

    def save_bm25_index(self):
        os.makedirs(os.path.dirname(self.bm25_path), exist_ok=True)
        with open(self.bm25_path, "wb") as f:
            pickle.dump(self.bm25, f)
        logging.info(f"✅ BM25 인덱스 저장 완료: {self.bm25_path}")

    def load_bm25_index(self):
        path = self.bm25_path
        if os.path.exists(path):
            try:
                with open(path, "rb") as f:
                    self.bm25 = pickle.load(f)
                self.bm25_ready = True
                logging.info(f"✅ BM25 인덱스 로드 완료: {path}")
            except Exception as e:
                self.bm25_ready = False
                logging.warning(f"❌ BM25 인덱스 로드 실패: {e}")
        else:
            self.bm25_ready = False
            logging.warning(f"❌ BM25 인덱스 파일 없음: {path}")

    def deduplicate_documents(self, documents: List[Document]) -> List[Document]:
        seen = set()
        unique_docs = []
        for doc in documents:
            chunk_id = doc.metadata.get("chunk_id")
            if chunk_id and chunk_id not in seen:
                seen.add(chunk_id)
                unique_docs.append(doc)
        removed = len(documents) - len(unique_docs)
        logging.info(f"🧹 중복 제거: {removed}개 제거됨")
        return unique_docs

    
    async def load_or_cache_json_docs(self, folder_path: str, cache_path: str) -> List[Document]:
        # 캐시 디렉토리 자동 생성
        os.makedirs(os.path.dirname(cache_path), exist_ok=True)
    
        if os.path.exists(cache_path):
            with open(cache_path, "rb") as f:
                logging.info("📦 캐시된 JSON 문서 로드 중...")
                return pickle.load(f)
        else:
            logging.info("📂 JSON 폴더에서 문서 로딩 중...")
            docs = await self.async_load_chunks_from_folder(folder_path)
            with open(cache_path, "wb") as f:
                pickle.dump(docs, f)
            logging.info("✅ JSON 캐시 저장 완료")
            return docs

    async def async_load_chunks_from_folder(self, folder_path: str) -> List[Document]:
        if not os.path.isdir(folder_path):
            raise ChunkLoadingError(f"❌ 폴더 경로가 존재하지 않음: {folder_path}")

        file_list = sorted([f for f in os.listdir(folder_path) if f.endswith(".json")])
        existing_sources = self.get_existing_chunk_ids()
        logging.info(f"📁 기존 DB에 저장된 source 수: {len(existing_sources)}")

        tasks = []
        for filename in file_list:
            if filename in existing_sources:
                logging.info(f"⏩ 이미 처리된 파일 건너뜀: {filename}")
                continue
            file_path = os.path.join(folder_path, filename)
            tasks.append(self._load_single_file(file_path, filename))

        # ✅ 고급 tqdm 적용: 실제 완료 기준으로 진행률 표시
        all_chunks = []
        for coro in tqdm_asyncio.as_completed(tasks, desc="📂 파일 처리 중", total=len(tasks)):
            result = await coro
            all_chunks.append(result)

        documents = [doc for sublist in all_chunks for doc in sublist]
        logging.info(f"✅ 새로 로드된 문서 수: {len(documents)}")

        documents = self.deduplicate_documents(documents)
        logging.info(f"🧹 중복 제거 후 문서 수: {len(documents)}")
        return documents

    
    async def _load_single_file(self, file_path: str, filename: str) -> list[Document]:
        try:
            async with aiofiles.open(file_path, "r", encoding="utf-8") as f:
                content = await f.read()
                data = json.loads(content)
    
            metadata_base = data.get("csv_metadata", {})
            docs = []
    
            for idx, page in enumerate(data.get("pdf_data", [])):
                page_num = page.get("page", idx)
                text = page.get("text", "").strip()
    
                if text:
                    metadata = metadata_base.copy()
                    metadata["chunk_id"] = f"{filename}::page::{page_num}::type::text"
                    metadata["page"] = page_num
    
                    docs.append(Document(page_content=text, metadata=metadata))
    
            return docs
    
        except Exception as e:
            logging.warning(f"⚠️ 파일 로딩 실패: {filename} | 오류: {e}")
            return []

    def get_all_documents_from_db(self) -> List[Document]:
        if self.db is None:
            raise ValueError("Vector DB가 초기화되지 않았습니다.")

        all_docs = []
        collection = self.db._collection
        count = collection.count()
        offset = 0
        limit = 1000

        while offset < count:
            results = collection.get(
                limit=limit,
                offset=offset,
                include=["metadatas", "documents"]
            )
            for doc, meta in zip(results["documents"], results["metadatas"]):
                if not meta.get("chunk_id"):
                    logging.warning("⚠️ chunk_id 누락된 문서 발견")
                all_docs.append(Document(page_content=doc, metadata=meta))
            offset += limit

        return all_docs


    def load_or_build_vector_db(self, documents: List[Document], force_rebuild: bool = False):
        """Vector DB와 BM25 인덱스를 로드하거나 새로 구축합니다."""
        
        # 1단계: Vector DB 초기화 (로드 또는 생성)
        self._initialize_vector_db(documents, force_rebuild)
        
        # 2단계: 새 문서가 있다면 DB에 추가
        new_docs = self._update_db_with_new_docs(documents)
        
        # 3단계: BM25 인덱스 동기화 (새 문서 추가 여부에 따라 처리)
        self.documents = self.get_all_documents_from_db()
        self._synchronize_bm25_index(was_db_updated=(len(new_docs) > 0))
        
        # 4단계: 리랭커 임베딩 캐싱
        self._cache_reranker_embeddings()
        logging.info("🚀 모든 DB 및 인덱스 준비 완료.")
    

    def _initialize_vector_db(self, documents: List[Document], force_rebuild: bool):
        """Vector DB를 생성하거나 로드합니다."""
        if force_rebuild or not self._db_exists():
            logging.info("🆕 벡터 DB 생성 중...")
            wrapped_docs = list(tqdm(documents, desc="🔄 문서 임베딩 중"))
            self.db = Chroma.from_documents(wrapped_docs, self.embedder, persist_directory=self.persist_directory)
            logging.info("✅ 새 DB 구축 완료.")
        else:
            logging.info("✅ 기존 벡터 DB 로드 중...")
            self.load_vector_db()
    
    def _update_db_with_new_docs(self, documents: List[Document]) -> List[Document]:
        """새로운 문서를 필터링하여 DB에 추가합니다."""
        new_docs = self._filter_new_documents(documents)
        if not new_docs:
            logging.info("⏩ 새 문서 없음, DB 추가 생략")
            return []
    
        logging.info(f"➕ 새 문서 {len(new_docs)}개 추가 중...")
        batch_size = 100
        total_batches = (len(new_docs) + batch_size - 1) // batch_size
        
        for batch in tqdm(chunked(new_docs, batch_size), desc="📦 배치 추가 중", total=total_batches):
            self.db.add_documents(batch)
            
        return new_docs
    
    def _synchronize_bm25_index(self, was_db_updated: bool):
        """DB 상태에 따라 BM25 인덱스를 생성하거나 로드합니다."""
        # 새 문서가 추가됐다면 BM25 인덱스는 무조건 새로 만들어야 함
        if was_db_updated:
            logging.info("🔧 새 문서 추가됨, BM25 인덱스 재생성...")
            self.build_bm25_index()
            self.save_bm25_index()
            logging.info("✅ BM25 인덱싱 완료.")
            return
    
        # 새 문서가 없다면, 기존 인덱스를 로드해보고 없으면 생성
        if not hasattr(self, "bm25_ready") or not self.bm25_ready:
            self.load_bm25_index()
            if not self.bm25_ready:
                logging.info("📚 기존 BM25 인덱스 없음, 새로 구축 시작...")
                self.build_bm25_index()
                self.save_bm25_index()
                logging.info("✅ BM25 인덱싱 완료.")
    
    def _cache_reranker_embeddings(self):
        """리랭커 모델을 위한 임베딩을 캐싱합니다."""
        if not self.documents:
            logging.warning("⚠️ 캐싱할 문서가 없습니다.")
            return
            
        logging.info("💡 리랭커 임베딩 캐싱 중...")
        texts = [doc.page_content[:self.rerank_max_length] for doc in self.documents]
        self.reranker.cache_embeddings(texts, max_length=self.rerank_max_length)
        logging.info("✅ 리랭커 캐싱 완료.")

    def load_vector_db(self):
        if not self.persist_directory or not os.path.exists(self.persist_directory):
            raise ValueError("저장된 DB가 없거나 persist_directory가 잘못 설정되었습니다.")
        self.db = Chroma(persist_directory=self.persist_directory, embedding_function=self.embedder)

    def _db_exists(self) -> bool:
        if not self.persist_directory:
            return False
        required_files = ["chroma.sqlite3"]
        return all(os.path.exists(os.path.join(self.persist_directory, f)) for f in required_files)

    def get_existing_chunk_ids(self) -> Set[str]:
        if self.db is None:
            try:
                self.load_vector_db()
            except Exception as e:
                logging.warning(f"⚠️ DB 로드 실패: {e}")
                self.db = None
                return set()

        try:
            result = self.db.get(include=["metadatas"])
            chunk_ids = set()
            for i, meta in enumerate(result.get("metadatas", [])):
                cid = meta.get("chunk_id")
                if not cid:
                    logging.warning(f"⚠️ chunk_id 누락된 문서 발견 (index={i})")
                    continue
                chunk_ids.add(cid)
            return chunk_ids
        except Exception as e:
            logging.warning(f"⚠️ chunk_id 목록 추출 실패: {e}")
            return set()


    def _filter_new_documents(self, documents: List[Document]) -> List[Document]:
        existing_chunk_ids  = self.get_existing_chunk_ids()
        new_docs = []
        for doc in documents:
            chunk_id  = doc.metadata.get("chunk_id")
            if chunk_id  and chunk_id  not in existing_chunk_ids :
                new_docs.append(doc)
        return new_docs

    # ✅ BM25 관련 함수 추가
    def build_bm25_index(self):
        if self.bm25 is not None:
            logging.info("⏩ BM25 인덱스 이미 존재함, 재생성 생략")
            return
        if not self.documents:
            raise ValueError("BM25 인덱스를 생성할 문서가 없습니다.")

        tokenized_corpus = [self.tokenizer.tokenize_korean(doc.page_content) 
                            for doc in tqdm(self.documents, desc="🧠 문서 토큰화 중")]
        self.bm25 = BM25Okapi(tokenized_corpus)
        self.bm25_ready = True
        logging.info("✅ BM25 인덱스 생성 완료")

    def _debug_print_bm25_scores(self, top_docs: List[Tuple[float, Document]]):
        """BM25 점수 디버그 출력 전용 함수"""
        print("\n📈 BM25 상위 문서 및 점수:")
        for i, (score, doc) in enumerate(top_docs):
            print(f"BM25 문서 {i+1} | 점수: {score:.4f} | 출처: {doc.metadata.get('파일명')}")
            print(doc.page_content[:300])
            print("-" * 40)
        
    def bm25_search(self, query: str, k: int = 5, filter: Dict = None, debug: bool = False) -> List[Tuple[float, Document]]:
        if not self.bm25_ready:
            raise ValueError("BM25 인덱스가 준비되지 않았습니다.")

        tokenized_query = self.tokenizer.tokenize_korean(query)
        doc_scores = self.bm25.get_scores(tokenized_query)
           
        # ✅ 필터링 적용
        filtered_docs = []
        for score, doc in zip(doc_scores, self.documents):
            if filter:
                match = all(doc.metadata.get(k) == v for k, v in filter.items())
                if not match:
                    continue
            filtered_docs.append((score, doc))

        top_docs = sorted(filtered_docs, key=lambda x: x[0], reverse=True)[:k]

        if self.debug_mode:
            self._debug_print_bm25_scores(top_docs)
            
        return top_docs

    def rerank_documents(self, query: str, documents: List[Document]) -> Dict[str, float]:
        """쿼리와 문서들의 유사도를 계산하여 재순위화 점수 반환"""
        if not self.reranker:
            logging.warning("⚠️ 재순위화 모델이 로드되지 않았습니다. 점수가 0으로 반환됩니다.")
            return {self.get_doc_key(doc): 0.0 for doc in documents}
        
        texts_to_rerank = [doc.page_content[:self.rerank_max_length] for doc in documents]
        base_scores = self.reranker.rerank(query, texts_to_rerank)
    
        query_tokens = self.tokenizer.tokenize_korean(query)
        bonus_weight = 1.0
    
        final_scores = {}
        for doc, base_score in zip(documents, base_scores.values()):
            bonus = 0
            if query in doc.page_content:
                bonus += 2.0
            bonus += sum(1 for token in query_tokens if token in doc.page_content) * bonus_weight
    
            key = self.get_doc_key(doc)
            final_scores[key] = base_score + bonus
    
        logging.info("🧠 재순위화 점수 계산 완료 (보너스 포함)")
        return final_scores
    
    def _calculate_combined_scores(self, documents: List[Document], query: str, 
                                 bm25_scores: Dict[str, float], rerank_scores: Dict[str, float]) -> List[Tuple[float, Document]]:
        """문서들에 대한 BM25 + 재순위화 점수를 계산하여 반환"""
        final_scored = []
        self.last_scores= {}

        # 1. 점수 정규화
        scaled_bm25 = minmax_scale(bm25_scores)
        scaled_rerank = minmax_scale(rerank_scores)

        for doc in documents:
            key = self.get_doc_key(doc)
            bm25 = scaled_bm25.get(key, 0.0)
            rerank = scaled_rerank.get(key, 0.0)
            combined = self.bm25_weight * bm25 + self.rerank_weight * rerank
            
            self.last_scores[key] = {
                "bm25": bm25,
                "rerank": rerank,
                "combined": combined
            }
            final_scored.append((combined, doc))
    
        return final_scored

    def _debug_print_scores(self, documents: List[Document], search_type: str):
        """디버그 모드일 때 점수 정보 출력"""
        if not self.debug_mode:
            return
            
        logging.info(f"{search_type} 점수 정보")
        for doc in documents:
            key = self.get_doc_key(doc)
            scores = self.last_scores.get(key, {})
            bm25 = scores.get("bm25", 0.0)
            rerank = scores.get("rerank", 0.0)
            combined = scores.get("combined", 0.0)

            source = doc.metadata.get("파일명", "❓")
            chunk_index = doc.metadata.get("chunk_id", "❓")
            print(f"🔍 {source} | Chunk {chunk_index} | BM25: {bm25:.2f} | Rerank: {rerank:.2f} | Combined: {combined:.2f}")

        
    def _merge_search_results(self, vector_results: List[Document], bm25_results: List[Tuple[float, Document]]) -> Tuple[List[Document], Dict[str, float]]:
        """벡터와 BM25 검색 결과를 병합하고 점수 딕셔너리 반환"""
        merged = {}
        bm25_scores = {}
        
        # BM25 결과 우선 추가
        for score, doc in bm25_results:
            key = self.get_doc_key(doc)
            merged[key] = doc
            bm25_scores[key] = score
        
        # 벡터 결과에서 새로운 문서만 추가
        for doc in vector_results:
            key = self.get_doc_key(doc)
            if key not in merged:
                merged[key] = doc
                bm25_scores[key] = 0.0
        
        return list(merged.values()), bm25_scores
    
    def hybrid_search(self, query: str, top_k: int = 3, candidate_size: int = 10,
                      filter_dict: Dict = None, candidate_filenames: List[str] = None) -> List[Document]:
        """하이브리드 검색: 벡터 + BM25 + 재순위화 + 필터링"""
        if self.db is None:
            raise ValueError("Vector DB가 초기화되지 않았습니다.")
        if not self.bm25_ready:
            raise ValueError("BM25 인덱스가 준비되지 않았습니다.")
    
        if filter_dict:
            logging.info(f"🔍 하이브리드 필터 적용: {filter_dict}")
    
        # 1. 단순 필터 (=)만 추출
        simple_filter = {
            key: val["value"]
            for key, val in filter_dict.items()
            if val.get("operator") == "=" and key != "사업 요약"
        } or None
    
        # 2. 벡터 + BM25 검색
        vector_results = self.db.similarity_search(query, k=candidate_size, filter=simple_filter)
        bm25_results = self.bm25_search(query, k=candidate_size, filter=simple_filter)
        
        # 3. 파일명 기반 후보 제한 (metadata 쿼리일 때만 적용됨)
        if candidate_filenames:
            vector_results = [doc for doc in vector_results if doc.metadata.get("파일명") in candidate_filenames]
            bm25_results = [(score, doc) for score, doc in bm25_results if doc.metadata.get("파일명") in candidate_filenames]
            logging.info(f"📁 파일명 기반 후보 제한 적용됨: {len(candidate_filenames)}개")

        # 4. 결과 병합
        merged_docs, bm25_scores = self._merge_search_results(vector_results, bm25_results)
    
        # 5. 고급 조건 필터링 (>, < 등)
        filtered_docs = [doc for doc in merged_docs if check_filter_match(doc.metadata, filter_dict)]
        logging.info(f"✅ 고급 필터링 후 문서 수: {len(filtered_docs)}")
    
        final_docs_to_score = filtered_docs if filtered_docs else merged_docs
        if not filtered_docs:
            logging.warning(f"⚠️ 필터링 결과 없음 → 원본 결과에서 상위 {top_k}개 반환")
    
        # 6. 재순위화
        rerank_scores = self.rerank_documents(query, final_docs_to_score)
    
        # 7. 최종 점수 계산 및 정렬
        scored_docs = self._calculate_combined_scores(final_docs_to_score, query, bm25_scores, rerank_scores)
        final_results = sorted(scored_docs, key=lambda x: x[0], reverse=True)
    
        if self.debug_mode:
            self._debug_print_scores([doc for _, doc in final_results], "하이브리드 검색")
    
        logging.info(f"📊 하이브리드 검색 완료: {len(final_results)} → {top_k}개 반환")
        return [doc for _, doc in final_results[:top_k]]


    def detect_query_type(self, query: str, filters: Dict[str, Dict]) -> str:
        normalized_query = query.replace(" ", "").lower()
        
        explicit_summary_keywords = normalize_keywords([
            "사업요약", "공고요약", "사업개요", "공고개요"
        ])
        
        metadata_keywords = normalize_keywords([
            "사업금액",  "입찰일", "입찰시작일", "참여시작일",
            "입찰마감일", "참여마감일", "공고번호", "공개일자", "입찰공고일"
        ])

        if any(k in normalized_query for k in explicit_summary_keywords):
            return "metadata"
        if any(filters for field in metadata_keywords):
            return "metadata"
        return "semantic"

    
    def smart_search(self, query: str, top_k: int = 5, candidate_size: int = 10) -> List[Document]:
        """스마트 검색: 필터 추출 + 쿼리 유형 판단 + 하이브리드 검색"""
        if self.db is None or not self.bm25_ready:
            raise ValueError("❌ Vector DB 또는 BM25 인덱스가 준비되지 않았습니다.")
    
        # 1. 필터 추출
        filters = extract_filters(query, self.meta_df, self.tokenizer)
        logging.info(f"🧠 추출된 필터: {filters}")
    
        # 2. 쿼리 유형 판단
        query_type = self.detect_query_type(query, filters)
    
        # 3. 발주기관 키워드 제거
        if filters.get("발주 기관"):
            agency_name = filters["발주 기관"]["value"]
            query = re.sub(rf"\b{re.escape(agency_name)}\b", "", query).strip()
            logging.info(f"🧹 쿼리에서 발주기관 키워드 제거됨: '{agency_name}'")
    
        # 4. 메타데이터 기반 필터링 (metadata 쿼리일 때만)
        matched_records = []
        candidate_filenames = None
        if query_type == "metadata" and self.meta_df is not None:
            matched_df = self.meta_df[
                self.meta_df.apply(lambda row: check_filter_match(row, filters), axis=1)
            ]
            logging.info(f"📊 메타데이터 필터링 완료: {len(matched_df)}개")
    
            if not matched_df.empty:
                matched_records = matched_df.head(10).to_dict(orient="records")
                candidate_filenames = matched_df["파일명"].dropna().unique().tolist()
                logging.info(f"📁 의미 검색 대상 제한됨 (파일명 기준): {len(candidate_filenames)}개")
            else:
                logging.warning("⚠️ 메타데이터 필터링 결과 없음 → 전체 문서 대상으로 검색")
    
        # 5. 하이브리드 검색 실행
        logging.info("🔍 의미 기반 하이브리드 검색 실행")
        semantic_docs = self.hybrid_search(
            query=query,
            top_k=top_k,
            candidate_size=candidate_size,
            filter_dict=filters,
            candidate_filenames=candidate_filenames
        )
        
        return matched_records, semantic_docs
        

In [9]:
import re
import logging
import pandas as pd
from datetime import datetime
from typing import List, Dict, Union, Any, Optional

FILTER_MAPPER = {
    "사업금액": {
        "field": "사업 금액", 
        "type": int,
        "pattern": r"(사업\s?금액)?\s*(\d+[억만천백조]+)\s*(이상|이하|초과|미만)?"
    },
    "입찰시작일": {
        "field": "입찰 참여 시작일",
        "type": "date",
        "pattern": r"(입찰\s?시작일|참여\s?시작일)[^\d]*(\d{4})[년\s]*(\d{1,2})?[월]?"
    },
    "입찰마감일": {
        "field": "입찰 참여 마감일",
        "type": "date",
        "pattern": r"(입찰\s?마감일|참여\s?마감일)[^\d]*(\d{4})[년\s]*(\d{1,2})?[월]?"
    },
    "입찰공고일": {
        "field": "공개 일자",
        "type": "date",
        "pattern": r"(입찰\s?공고일)[^\d]*(\d{4})[년\s]*(\d{1,2})?[월]?"
    },
    "발주기관": {
        "field": "발주 기관",  
        "type": str,
        "pattern": r"(한국농어촌공사|조달청|도로공사|[가-힣]{2,})"
    },
    "공고번호": {
        "field": "공고 번호", 
        "type": str,
        "pattern": r"(공고번호\s?\d{4}-?\d{3,})"
    },
}
def normalize_keywords(keywords: list[str]) -> set[str]:
    """키워드 리스트를 정규화하여 비교 가능하게 변환"""
    return {k.replace(" ", "").lower() for k in keywords}

def safe_parse_date(value: str) -> Optional[datetime]:
    if not isinstance(value, str):
        return None
    try:
        parts = [int(p) for p in re.findall(r"\d+", value)]
        if len(parts) >= 2:
            year, month = parts[0], parts[1]
            day = parts[2] if len(parts) > 2 else 1
            return datetime(year, month, day)
    except Exception as e:
        logging.warning(f"❌ 날짜 파싱 실패: {value} → {e}")
        return None

def parse_korean_number(text: str) -> int:
    unit_values = {
        "십": 10,
        "백": 100,
        "천": 1000,
        "만": 10_000,
        "억": 100_000_000,
        "조": 1_000_000_000_000
    }

    # 정규화
    text = text.replace(",", "").replace("억원", "억").replace("백만원", "백만") \
               .replace("천만원", "천만").replace("만원", "만").replace("원", "").strip()
    print("🌸 처리전 text:", text)

    # 단위별 블록 추출
    blocks = re.findall(r"(\d+)([십백천만억조]+)", text)

    total = 0
    current_block = 0
    last_big_unit = 1

    for num_str, unit_str in blocks:
        num = int(num_str)
        small_unit = 1
        big_unit = 1

        for char in unit_str:
            if char in ["십", "백", "천"]:
                small_unit *= unit_values[char]
            elif char in ["만", "억", "조"]:
                big_unit = unit_values[char]

        current_block += num * small_unit

        # 큰 단위가 붙었으면 전체 블록에 곱해서 total에 더함
        if big_unit > 1:
            total += current_block * big_unit
            current_block = 0

    total += current_block
    print("🌸 처리완료후:", total)
    return total


# 예시:
# parse_korean_number("5천만원")        # 50000000
# parse_korean_number("1억 2천만원")    # 120000000
# parse_korean_number("2천만")         # 20000000
# parse_korean_number("3백억")         # 30000000000
# parse_korean_number("456백만")       # 456000000


def convert_value(raw: str, value_type):
    """필터 추출용 값 변환"""
    if value_type in ("int", int):
        return parse_korean_number(raw)
    elif value_type in ("date", datetime):
        return safe_parse_date(raw)
    elif value_type in ("float", float):
        try:
            return float(str(raw).replace(",", "").strip())
        except ValueError:
            return None
    else:
        return raw.strip()


OPERATOR_FUNC = {
    ">=": lambda v, t: v >= t,
    "<=": lambda v, t: v <= t,
    ">":  lambda v, t: v > t,
    "<":  lambda v, t: v < t,
    "=":  lambda v, t: v == t,
    "~":  lambda v, t: abs(v - t) <= t * 0.1
}

def extract_operator(text: str, context: str = "") -> str:
    full_text = text + " " + context
    if "이후" in full_text or "부터" in full_text or "최소" in full_text or "이상" in full_text:
        return ">="
    elif "이전" in full_text or "까지" in full_text or "최대" in full_text or "이하" in full_text:
        return "<="
    elif "초과" in full_text:
        return ">"
    elif "미만" in full_text:
        return "<"
    elif "약" in full_text or "정도" in full_text:
        return "~"
    return "="


# 🚨 기관명/파일명 필터링 시 제거할 잡음 단어 목록
NOISE_WORDS = {
     # 날짜/시점 관련
    "년", "월", "일", "년도", "2024", "2025",

    # 입찰/공고 관련
    "입찰", "공고", "재공고", "긴급", "협상", "사전공개",

    # 금액 관련
    "원", "예산",

}

def extract_field_filter_by_tokens(
    query: str,
    field_values: List[str],
    tokenizer,
    field_name: str,
    use_exact_match: bool = True,
    threshold: float = 0.5
) -> Optional[Dict[str, Dict]]:
    
    query_tokens = set(tokenizer.tokenize_korean(query, use_bigrams=False)) - NOISE_WORDS
    print("❤️query_tokens", query_tokens)
    logging.debug(f"🧹 필터링용 토큰셋: {query_tokens}")

    # 1️⃣ 정확 매칭 우선
    if use_exact_match:
        for value in field_values:
            if value and value in query:
                return {field_name: {"value": value, "operator": "="}}

    # 유사도 기반 매칭
    best_match = None
    best_score = 0
    for value in field_values:
        match_count = sum(1 for token in query_tokens if token in value)
        score = match_count / len(query_tokens) if query_tokens else 0
        if score > best_score and score > threshold:
            best_match = value
            best_score = score
           
    if best_match:
        print("❤️best_match", best_match, best_score)
        return {field_name: {"value": best_match, "operator": "="}}

    return None


def extract_filters(query: str, meta_df: pd.DataFrame, tokenizer) -> Dict[str, Dict]:
    filters = {}
    
    # 3️⃣ 정규식 기반 필터 추출
    for keyword, filter_info in FILTER_MAPPER.items():
        field_name = filter_info.get("field")
    
        # ✅ 기관명/파일명은 정규식으로 추출하지 않음
        if field_name in filters or field_name in ["발주 기관", "파일명"]:
            continue
            
        match = re.search(filter_info['pattern'], query)
        if match:
            value_type = filter_info.get('type')
            if value_type == "date":
                year = match.group(2)
                month = match.group(3) if match.lastindex and match.lastindex >= 3 and match.group(3) else "1"
                raw_value = f"{year}년 {month}월"
            elif value_type == int:
                raw_value = match.group(2)
                condition = match.group(3) or ""
                operator = extract_operator(condition, query)
            else:
                raw_value = match.group(1)
                operator = "="
    
            value = convert_value(raw_value, value_type)
            operator = extract_operator(raw_value, query) if value_type in ["date", int] else "="
            if value is not None:
                filters[field_name] = {"value": value, "operator": operator}
                logging.info(f"📌 {field_name} 필터 적용됨: {value} ({operator})")

    # 발주 기관 필터링
    agency_filter_applied = False
    if "발주 기관" in meta_df.columns:
        agency_list = meta_df["발주 기관"].dropna().unique().tolist()
        agency_filter = extract_field_filter_by_tokens(
            query=query,
            field_values=agency_list,
            tokenizer=tokenizer,
            field_name="발주 기관",
            use_exact_match=True,
            threshold=0.5
        )
        
        print("❤️agency_filter : ", agency_filter)
        if agency_filter:
            filters.update(agency_filter)
            agency_filter_applied = True
            logging.info(f"🏢 발주 기관 필터 적용됨: {agency_filter['발주 기관']['value']}")
   
    # 파일명 보조 필터링 (기관 필터 없을 때만)
    if not agency_filter_applied and "파일명" in meta_df.columns:
        filename_list = meta_df["파일명"].dropna().unique().tolist()
        print("❤️파일필터작동 ")
        filename_filter = extract_field_filter_by_tokens(
            query=query,
            field_values=filename_list,
            tokenizer=tokenizer,
            field_name="파일명",
            use_exact_match=True,
            threshold=0.5  # ✅ 더 유연하게
        )
        print("❤️file_filter : ", filename_filter)
        if filename_filter:
            filters.update(filename_filter)
            logging.info(f"📁 파일명 필터 적용됨: {filename_filter['파일명']['value']}")

    return filters


def is_valid_value(value):
    """값이 유효한지 확인하는 헬퍼 함수"""
    if value is None or str(value).strip() in ["", "미정", "nan"]:
        return False
    return True

def check_filter_match(data: Union[Dict, Any], filters: Dict[str, Dict]) -> bool:
    for field, condition in filters.items():
        raw_value = data.get(field)

        if not is_valid_value(raw_value):
            return False

        target = condition["value"]
        operator = condition["operator"]

        # ✅ raw_value가 target과 같은 타입인지 확인
        try:
            if isinstance(target, int):
                value = int(str(raw_value).replace(",", "").strip())
            elif isinstance(target, float):
                value = float(str(raw_value).replace(",", "").strip())
            elif isinstance(target, datetime):
                value = safe_parse_date(str(raw_value))
            else:
                value = str(raw_value).strip()
        except Exception:
            return False

        compare_func = OPERATOR_FUNC.get(operator)
        if not compare_func:
            return False

        try:
            return compare_func(value, target)
        except TypeError:
            return False

    return True

# 예시: '사업금액 5천만원 이상인 공고 찾아줘'

In [47]:
from dotenv import load_dotenv

def merge_docs_to_text(docs):
    """
    의미 기반 검색 결과(Document 리스트)를 받아서
    LLM context로 합치는 함수.
    (문서 전체 내용을 합침, 잘라내기 제한 없음)
    """
    if not docs:
        return ""

    merged = []
    for doc in docs:
        text = doc.page_content.strip()
        merged.append(f"[출처: {doc.metadata.get('파일명', '❓')}]\n{text}")

    return "\n\n".join(merged)


from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

# .env 파일 로드
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY, 
    model_name="gpt-4o-mini",
    temperature=1,
)

prompt = ChatPromptTemplate.from_template("""
다음은 사용자의 질문과 관련된 검색 결과입니다:

{context}

위 내용을 바탕으로 사용자의 질문에 대해 간결하고 정확하게 답변해주세요:
질문: {question}
""")

qa_chain = LLMChain(llm=llm, prompt=prompt)

In [48]:
import os
import sys
import asyncio
import logging
import pandas as pd
import time

from config import Config
from langchain_huggingface import HuggingFaceEmbeddings
from sentence_transformers import CrossEncoder

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    datefmt="%H:%M:%S"
)

if __name__ == "__main__":
    # 설정 로드
    cfg = Config()
    
    # 기본 설정
    LOAD_MODE = "json"
    TOKENIZER = "kiwi"
    
    # 메타데이터 로드
    meta_df = pd.read_csv(cfg.meta_csv_path)
    
    # 디바이스 설정
    import torch
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"✅ 디바이스: {device}")
    
    # 모델 초기화 (외부에서 로드하여 캐시화)
    embedder = HuggingFaceEmbeddings(
        model_name=cfg.embedder_model,
        model_kwargs={"device": device}
    )

    reranker = RerankModel(
        model_name=cfg.reranker_model,
        cache_dir=cfg.rerank_cache_dir,
        device=device
    )
        
    tokenizer = TokenizerWrapper(TOKENIZER)
    
    # Retriever 초기화 
    retriever = Retriever(
        meta_df=meta_df,
        embedder=embedder,
        reranker=reranker,
        tokenizer=tokenizer,
        persist_directory=cfg.chroma_db_path,
        rerank_max_length=cfg.rerank_max_length,
        bm25_path=cfg.bm25_path,
        debug_mode=True
    )
    
    try:
        # 문서 로딩 (기존 로직 100% 보존)
        if LOAD_MODE == "json":
            docs = await retriever.load_or_cache_json_docs(
                cfg.json_dir, 
                cache_path=cfg.cached_json_path
            )
        
        # 가중치 설정 및 벡터 DB 구축 (기존과 동일)
        retriever.set_weights(bm25_weight=0.3, rerank_weight=0.7)
        retriever.load_or_build_vector_db(docs)
        
        # 검색 실행
        query_text = "서울특별시 공공데이터 개방이나 연계 관련해서 어떤 요구사항이 있어?"
        logging.info(f"\n[스마트 검색 실행] 질문: {query_text}")
        
        tokenized_query = retriever.tokenizer.tokenize_korean(query_text)
        print(f"\n🔍 BM25 키워드 토큰: {tokenized_query}")
        
        matched_records, semantic_docs = retriever.smart_search(
            query=query_text,
            top_k=3,
            candidate_size=10,
        )

        # LLM 응답 생성
        logging.info("🧠 LLM 응답 생성 시작중...")
        start_time = time.time()
        
        merged_text = merge_docs_to_text(semantic_docs)
        response = qa_chain.invoke({
            "context": merged_text,
            "question": query_text
        })
       
        final_answer = response.get("text", "").strip()      
        print("\n🧠 LLM 응답 생성 응답:")
        print(final_answer)
        
        elapsed = time.time() - start_time
        logging.info(f"🧠 LLM 응답 생성 완료 (소요 시간: {elapsed:.2f}초)")
        
        print("\n📈 반환 문서 내용:")
        
        # 참고용 메타데이터 출력 (최대 10개)
        if matched_records:
            print("\n📊 메타데이터 필터링 결과 (최대 10개):")
            for i, record in enumerate(matched_records, 1):
                print(f"\n📄 문서 {i}")
                for key, value in record.items():
                    print(f"🔹 {key}: {value}")
                print("=" * 50)
        
        # 의미 기반 검색 결과
        for i, doc in enumerate(results, 1):
            key = retriever.get_doc_key(doc)
            scores = retriever.last_scores.get(key, {})
            bm25 = scores.get("bm25", 0.0)
            rerank = scores.get("rerank", 0.0)
            combined = scores.get("combined", 0.0)
    
            print(f"문서 {i} | 출처: {doc.metadata.get('chunk_id')}")
            print(f"🔹 BM25 점수: {bm25:.2f} | 🔹 Rerank 점수: {rerank:.2f} | 🔹 Combined: {combined:.2f}")
            print(doc.page_content[:500])
            print("=" * 50)

    except Exception as e:
        logging.error(f"오류: {e}")


2025-09-26 08:41:54,096 - INFO - Load pretrained SentenceTransformer: nlpai-lab/KURE-v1


✅ 디바이스: cuda:0


2025-09-26 08:41:58,396 - INFO - 📦 캐시된 JSON 문서 로드 중...
2025-09-26 08:41:58,492 - INFO - 🔧 가중치 설정됨 | BM25: 0.3 | Rerank: 0.7
2025-09-26 08:41:58,493 - INFO - ✅ 기존 벡터 DB 로드 중...
2025-09-26 08:41:59,400 - INFO - ⏩ 새 문서 없음, DB 추가 생략
2025-09-26 08:42:01,306 - INFO - ✅ BM25 인덱스 로드 완료: /home/spai0320/projectmission2/data/cache/bm25_index.pkl
2025-09-26 08:42:01,307 - INFO - 💡 리랭커 임베딩 캐싱 중...
2025-09-26 08:42:01,387 - INFO - ✅ 리랭커 캐싱 완료.
2025-09-26 08:42:01,390 - INFO - 🚀 모든 DB 및 인덱스 준비 완료.
2025-09-26 08:42:01,390 - INFO - 
[스마트 검색 실행] 질문: 서울특별시 공공데이터 개방이나 연계 관련해서 어떤 요구사항이 있어?


✅ 의미 임베딩 캐싱 완료 | 새로 캐싱: 0개 | 스킵: 7569개


2025-09-26 08:42:03,095 - INFO - 🏢 발주 기관 필터 적용됨: 서울특별시
2025-09-26 08:42:03,098 - INFO - 🧠 추출된 필터: {'발주 기관': {'value': '서울특별시', 'operator': '='}}
2025-09-26 08:42:03,100 - INFO - 🧹 쿼리에서 발주기관 키워드 제거됨: '서울특별시'
2025-09-26 08:42:03,103 - INFO - 📊 메타데이터 필터링 완료: 1개
2025-09-26 08:42:03,106 - INFO - 📁 의미 검색 대상 제한됨 (파일명 기준): 1개
2025-09-26 08:42:03,107 - INFO - 🔍 의미 기반 하이브리드 검색 실행
2025-09-26 08:42:03,107 - INFO - 🔍 하이브리드 필터 적용: {'발주 기관': {'value': '서울특별시', 'operator': '='}}
2025-09-26 08:42:03,218 - INFO - 📁 파일명 기반 후보 제한 적용됨: 1개
2025-09-26 08:42:03,220 - INFO - ✅ 고급 필터링 후 문서 수: 13
2025-09-26 08:42:03,239 - INFO - 🧠 재순위화 점수 계산 완료 (보너스 포함)
2025-09-26 08:42:03,241 - INFO - 하이브리드 검색 점수 정보
2025-09-26 08:42:03,242 - INFO - 📊 하이브리드 검색 완료: 13 → 3개 반환
2025-09-26 08:42:03,242 - INFO - 🧠 LLM 응답 생성 시작중...



🔍 BM25 키워드 토큰: ['서울특별시', '공공', '데이터', '개방', '연계', '관련', '요구', '사항', '서울특별시공공', '공공데이터', '데이터개방', '개방연계', '연계관련', '관련요구', '요구사항']
❤️query_tokens {'공공', '사항', '데이터', '개방', '요구', '서울특별시', '관련', '연계'}
❤️agency_filter :  {'발주 기관': {'value': '서울특별시', 'operator': '='}}

📈 BM25 상위 문서 및 점수:
BM25 문서 1 | 점수: 32.0440 | 출처: 서울특별시_2024년 지도정보 플랫폼 및 전문활용 연계 시스템 고도화 용.pdf
- 34 - 요구사항번호 DAR-009 요구사항 명 데이터 관리체계 수립 요구사항 분류 데이터 품질 상세 설명 정의 데이터 관리체계 수립 세부 내용 데이터 관리체계 정의 - 표준, 구조, 연계 등 데이터 핵심 요소의 지속적인 관리를 위한 조직과 역할을 정의하여야 함 - 데이터 값의 진단 및 개선 등 품질관리 조직과 역할을 정의하여야 함 - 데이터 값 진단 방안기능, 진단 프로그램 등을 제시하여야 함 - 데이터 표준단어용어도메인코드의 추가, 변경, 삭제 절차를 수립하여야 함. - 데이터 구조논리물리와 관련된 추가, 변경, 삭제 절차를
----------------------------------------
BM25 문서 2 | 점수: 17.1946 | 출처: 서울특별시_2024년 지도정보 플랫폼 및 전문활용 연계 시스템 고도화 용.pdf
- 18 - 3. 제안요청 내용 가. 요구사항 구성 요구사항 분류 설 명 요구사항 번호ID부여 규칙 요 구 사항수 시스템 장비구성ECR Equipment Composition Requirement - 목표시스템의 구성을 위해 필요한 하드웨어, 소프트웨어, 네트워크 등의 도입 장비 내역 등 시스템 장비 구성에 대한 요구사항 1 기능SFR System Function Requirement - 목표시스템

2025-09-26 08:42:08,957 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-26 08:42:08,971 - INFO - 🧠 LLM 응답 생성 완료 (소요 시간: 5.73초)



🧠 LLM 응답 생성 응답:
서울특별시 공공데이터 개방 및 연계 관련 요구사항은 다음과 같습니다:

1. **개방 데이터 관리체계**:
   - 개방 데이터 서비스의 연속성을 확보하고, 관련 개방 데이터 목록을 식별해야 합니다.
   - 공공데이터 제공 및 이용 활성화를 위한 법률 및 조례를 준수하며, 개방대상 데이터에 대해 시스템 DB 연계 등 관련 업무를 지원해야 합니다.

2. **연계 데이터 관리체계**:
   - 제공기관과 활용기관 간의 연계 데이터 정합성을 유지하기 위한 방안을 제시해야 하며, 메타데이터를 작성하고 표준화해야 합니다.
   - 연계 데이터 품질 확보를 위해 협의체를 구성하고 정기적인 데이터 정합성 검증을 실시해야 합니다.

3. **데이터 표준화 및 관리**:
   - 데이터 표준 원칙 및 가이드를 수립하고, 범정부 및 서울시 표준을 준수해야 합니다.
   - 데이터 표준 관리 방안을 마련하여 메타데이터 관리 시스템 등을 통해 변경 이력을 효과적으로 관리해야 합니다.

이와 같은 요구사항을 충족함으로써 서울특별시의 공공데이터 개방과 연계 체계를 효과적으로 구축할 수 있습니다.

📈 반환 문서 내용:

📊 메타데이터 필터링 결과 (최대 10개):

📄 문서 1
🔹 공고 번호: 20240404154
🔹 공고 차수: 0.0
🔹 사업명: 2024년 지도정보 플랫폼 및 전문활용 연계 시스템 고도화 용역
🔹 사업 금액: 493763000
🔹 발주 기관: 서울특별시
🔹 공개 일자: 2024-04-02 15:49:39
🔹 입찰 참여 시작일: 2024-04-19 09:00:00
🔹 입찰 참여 마감일: 2024-04-23 16:00:00
🔹 사업 요약: - 사업개요: 2024년 지도정보 플랫폼 및 전문활용 연계 시스템 고도화 구축
- 추진배경 및 필요성: 외래관광객 유치, 개인화 및 로컬화된 관광 트렌드에 맞는 서비스 필요
- 사업범위: 매력서울지도 다국어 서비스 구축, 지도정보 플랫폼 고도화, 시각화 서비스 및 주소-좌표 변환 기능 고도화
- 기대

In [25]:
# 검색 실행 (기존 로직 그대로)

#query_text = "입찰시작일이 2024년 8월 이후 공고 찾아줘"
query_text = "사업금액 100억이상 공고 알려줘"
logging.info(f"\n[스마트 검색 실행] 질문: {query_text}")


tokenized_query = retriever.tokenizer.tokenize_korean(query_text)
print(f"\n🔍 BM25 키워드 토큰: {tokenized_query}")

results = retriever.smart_search(
    query=query_text,
    top_k=3,
    candidate_size=10,
)

# 결과 출력 (기존 로직 100% 유지)
print("\n📈 최종 결과:")

if results and isinstance(results[0], dict):
    # 메타데이터 기반 검색 결과
    for i, record in enumerate(results):
        print(f"\n📄 문서 {i+1}")
        for key, value in record.items():
            print(f"🔹 {key}: {value}")
        print("=" * 50)
else:
    # 의미 기반 검색 결과
    for i, doc in enumerate(results):
        key = retriever.get_doc_key(doc)
        scores = retriever.last_scores.get(key, {})
        bm25 = scores.get("bm25", 0.0)
        rerank = scores.get("rerank", 0.0)
        combined = scores.get("combined", 0.0)

        print(f"문서 {i+1} | 출처: {doc.metadata.get('chunk_id')}")
        print(f"🔹 BM25 점수: {bm25:.2f} | 🔹 Rerank 점수: {rerank:.2f} | 🔹 Combined: {combined:.2f}")
        print(doc.page_content[:500])
        print(doc.metadata)
        print("=" * 50)

2025-09-26 02:43:04,582 - INFO - 
[스마트 검색 실행] 질문: 사업금액 100억이상 공고 알려줘
2025-09-26 02:43:04,587 - INFO - 📌 사업 금액 필터 적용됨: 10000000000 (>=)
2025-09-26 02:43:04,592 - INFO - 🧠 추출된 필터: {'사업 금액': {'value': 10000000000, 'operator': '>='}}
2025-09-26 02:43:04,594 - INFO - 🔍 의미 기반 하이브리드 검색 실행
2025-09-26 02:43:04,595 - INFO - 🔍 하이브리드 필터 적용: {'사업 금액': {'value': 10000000000, 'operator': '>='}}
2025-09-26 02:43:04,671 - INFO - ✅ 고급 필터링 후 문서 수: 0
2025-09-26 02:43:04,673 - WARNING - ⚠️ 필터링 결과가 없습니다. 원본 결과에서 상위 {top_k}개 반환합니다.
2025-09-26 02:43:04,724 - INFO - 🧠 재순위화 점수 계산 완료 (보너스 포함)
2025-09-26 02:43:04,725 - INFO - 하이브리드 검색 점수 정보
2025-09-26 02:43:04,726 - INFO - 📊 하이브리드 검색 완료: 20 → 3개 반환



🔍 BM25 키워드 토큰: ['사업', '금액', '이상', '공고', '사업금액', '금액이상', '이상공고']
🌸 처리전 text: 100억
🌸 처리완료후: 10000000000
❤️query_tokens {'금액', '사업', '이상'}
❤️agency_filter :  None
❤️파일필터작동 
❤️query_tokens {'금액', '사업', '이상'}
❤️file_filter :  None

📈 BM25 상위 문서 및 점수:
BM25 문서 1 | 점수: 23.7156 | 출처: 세종테크노파크_세종테크노파크 인사정보 전산시스템 구축 용역 입찰공.pdf
- 45 - 새로운 신용평가등급이 없는 경우에는 합병 대상업체 중 가장 낮은 신용평가등급을 받은 업체의 신용평가등급으로 평가 2 유사사업 수행실적 평가 배점 기준 및 유사용역 기준은 아래와 같음 항목 계산 방법 배점한도 기 준 배점 유사 사업 실적 건수 준공실적 건수 4 A. 사업수행 5건 이상 4 B. 사업수행 4건 이상 3 C. 사업수행 3건 이상 2 D. 사업수행 2건 이상 1 항목 계산 방법 배점한도 기 준 배점 유사 사업 실적 금액 본사업비 준공실적금액 3 A. 200이상 3 B. 100이상 - 200미만 2 C. 100미
----------------------------------------
BM25 문서 2 | 점수: 21.7840 | 출처: 수협중앙회_강릉어선안전조업국 상황관제시스템 구축.pdf
- 59 - 별첨 1 기술평가객관적 평가기준 및 채점표 평 가 위 원 성 명 서명 제안업체명 가. 경영상태 평가항목 평 가 요 소 배점 기준 경영상태 회사채에 대한 신용평가등급 AAA 5.0 AA, AA0, AA- 4.5 A, A0, A- 4.0 BBB 3.5 BBB0, BBB- 3.0 BB, BB0 2.5 BB- 2.0 B, B0 1.5 B- 1.0 CCC 이하 0.5 나. 유사용역실적 평가항목 평 가 요 소 배점 기준 유사 분야에서의 유지보수 경험 용역 또는 구축 경험실적 최근3년 유사사업 